# Advent of Code 2024: Day 24
https://adventofcode.com/2024/day/24


## Part 1
Find how many paths will cross in the test area, only X and Y coordinates

### Get the data into a list of lists of tuples

In [1]:
myfile = open('input.txt', 'r')
data = myfile.read()
data_list = data.split('\n')
data_list = [[tuple(map(int, x.split(', '))) for x in line.split(' @ ')] for line in data_list]

### Create function to return the parameters of a line from a given position and velocity
Return **a** and **b** from the equation **y=ax+b**

In [2]:
def getLine(pos, vel):
  a = vel[1]/vel[0]
  b = pos[1] - a*pos[0]
  return a,b

### Create function to return the position where two lines intersect, if they do

In [3]:
def getIntersection(line1, line2):
  left = line1[0] - line2[0]
  right = line2[1] - line1[1]
  if left == 0:
    return None, None
  x = right / left
  y = line1[0]*x + line1[1]
  return x,y

### Create function to check if an intersection occurs within the valid area and forward in time from the initial positions

In [4]:
def validIntersection(x,y,lower,upper, pos1, vel1, pos2, vel2):
  if x is None and y is None:
    return False
  check1 = False
  check2 = False
  if (x >= lower) and (x <= upper) and (y >= lower) and(y <= upper):
    if vel1[0] < 0:
      if x < pos1[0]:
        check1 = True
    elif vel1[0] > 0:
      if x > pos1[0]:
        check1 = True
    else:
      if x == pos1[0]:
        check1 = True
    if vel2[0] < 0:
      if x < pos2[0]:
        check2 = True
    elif vel2[0] > 0:
      if x > pos2[0]:
        check2 = True
    else:
      if x == pos2[0]:
        check2 = True
  if check1 and check2:
    return True
  return False

### Create function to count valid intersections

In [5]:
def countIntersections(data, lower, upper):
  valids = 0
  for i, p1 in enumerate(data[:-1]):
    line1 = getLine(p1[0], p1[1])
    for p2 in data[i+1:]:
      line2 = getLine(p2[0],p2[1])
      x, y = getIntersection(line1, line2)
      if validIntersection(x,y,lower,upper, p1[0],p1[1],p2[0],p2[1]):
        valids += 1
  return valids

### Count valid intersections

In [6]:
l = 200000000000000
u = 400000000000000
countIntersections(data_list, l, u)

21679

## Part 2
Find the starting position of an object that will collide with every other object, using X, Y, and Z coordinates

### Use the Z3 solver

Give it the 6 parameters that need to found (starting positions and velocities). For each object in the data, give it an individual time parameter (as the object can colide with the objects in the data at different times).

Then add the constraints for each object in the data, such that there must be some time point _t_ where our object will collide with it.

In [7]:
from z3 import *

x, y, z, xv, yv, zv = Reals('x y z xv yv zv')
t = []
s = Solver()
for i, line in enumerate(data_list):
  p,v = line
  t.append(Real('t'+str(i)))
  s.add(x + xv*t[i] == p[0]+v[0]*t[i])
  s.add(y + yv*t[i] == p[1]+v[1]*t[i])
  s.add(z + zv*t[i] == p[2]+v[2]*t[i])
s.check()
model = s.model()

model.evaluate(x+y+z)

566914635762564